## Order of operations

1. Gadid specimen get in CLAMS2ABL based on genetic IDs provided by Sharon Wildes (ArcticEIS repo, specimenCorrections modelLen.py)
2. Run MBA for 201701 and 201901
3. Export the following tables:
    - catchData
        - mbaCatchResults (mbaCatchResults.sql)
        - v_aeis_catch_summary_v2
        - v_aeis_events
        - v_aeis_specimen
            - Build trawl selectivity tables:
                - Add consistent lengths for all gadid specimen (Arctic EIS repo, specimenCorrections addCommonLength.py)
                - Export Basket and sample tables from CLAMS2ABL:
                    `select * from baskets where (ship = 175 and (survey = 201701 or survey = 201901)) or (ship = 174)`
                    `select * from samples where (ship = 175 and (survey = 201701 or survey = 201901)) or (ship = 174)`
                - Add expansion values to the table (Arctic EIS repo, trawlSelectivity buildSelectivityTable.ipynb)
        - AIERP_EventData (ArcticEIS, EventExport.sql)
            - Calculate volme filtered (aeisEvents EventData.ipynb)
    - acousticData
        - mbaIntegrationResults (mbaResultsExport.sql)
        - mbaSpeciesProp (see SQL code below)

### Methods

In [2]:
# Transit speed
import pandas as pd
from glob import glob
import numpy as np

files = glob('../data/acousticData/2017_2019/EV/2017/Echoview/exports/5m/*(intervals).csv')
a = pd.concat(map(lambda file: pd.read_csv(file), files))
a['Datetime_S'] = pd.to_datetime(a['Date_S'].astype(str)+a['Time_S'])
a['Datetime_E'] = pd.to_datetime(a['Date_E'].astype(str)+a['Time_E'])
a['Duration'] = (a.Datetime_E- a.Datetime_S).dt.total_seconds()
a['Dist'] = a['VL_end']-a['VL_start']
a['Speed'] = a.Dist*1852/a.Duration
print('2017 transit speed (m/s): ',a.Speed.mean())
print(sum((a.Dist)*(30/2)))

from glob import glob
files = glob('../data/acousticData/2017_2019/EV/2019/Echoview/exports/5m/*(intervals).csv')
a = pd.concat(map(lambda file: pd.read_csv(file), files))
a['Datetime_S'] = pd.to_datetime(a['Date_S'].astype(str)+' '+a['Time_S'])
a['Datetime_E'] = pd.to_datetime(a['Date_E'].astype(str)+' '+a['Time_E'])
a['Duration'] = (a.Datetime_E- a.Datetime_S).dt.total_seconds()
a['Dist'] = a['VL_end']-a['VL_start']
a['Speed'] = a.Dist*1852/a.Duration
print('2019 transit speed (m/s): ',a.Speed.mean())
print(sum((a.Dist)*(19)))

2017 transit speed (m/s):  3.3434855897494025
47206.29784499888
2019 transit speed (m/s):  3.420634502352993
49963.68210099984


In [34]:
# Calibration
# From the E:\ChukchiTimeSeries\data\acousticData\2017_2019\EV\2019\calibration\Ocean Starr EK60 cals AFSC.xslx spreadhseet

# 38 kHz 512
# 2017 
# Cal1: 25.23, -0.88
# Cal2: 25.00, -0.83
print('2017, 38 @ .512: ',round(10*np.log10(((10**(25.23/10))+(10**(25.00/10)))/2),3),round(10*np.log10(((10**(-.88/10))+(10**(-.83/10)))/2),3))

# 2019
# Cal1: 25.17, -0.84
# Cal2: 25.15, -0.76
print('2019, 38 @ .512: ',round(10*np.log10(((10**(25.17/10))+(10**(25.15/10)))/2),3),round(10*np.log10(((10**(-.84/10))+(10**(-.76/10)))/2),3))


print(1-(10**((25.117-.855)/10))/(10**((25.16-.8)/10)))

# 38 kHz 4
# 2017 (only 1 cal) 25.32,-0.24
print('2017, 38 @ 4.0: ',25.32,-0.24)

# 2019
# Cal1: 25.25, -0.28
# Cal2: 25.48, -0.28
print('2019, 38 @ 4.0: ',round(10*np.log10(((10**(25.25/10))+(10**(25.48/10)))/2),3),round(10*np.log10(((10**(-.28/10))+(10**(-.28/10)))/2),3))

print(1-(10**((25.32 -0.24)/10))/(10**((25.367 -0.28)/10)))

2017, 38 @ .512:  25.117 -0.855
2019, 38 @ .512:  25.16 -0.8
0.02231264103257835
2017, 38 @ 4.0:  25.32 -0.24
2019, 38 @ 4.0:  25.367 -0.28
0.0016105112976727565


In [36]:
# 120 kHz 512
# 2017 
# Cal1: 25.07, -0.41
# Cal2: 24.49, -0.51
print('2017, 120 @ .512: ',round(10*np.log10(((10**(25.07/10))+(10**(24.49/10)))/2),3),round(10*np.log10(((10**(-.41/10))+(10**(-.51/10)))/2),3))

# 2019
# Cal1: 25.39, -0.46
# Cal2: 24.48, -0.44
print('2019, 120 @ .512: ',round(10*np.log10(((10**(25.39/10))+(10**(24.48/10)))/2),3),round(10*np.log10(((10**(-.46/10))+(10**(-.44/10)))/2),3))

print(1-(10**((24.79 -0.46)/10))/(10**((24.959 -0.45)/10)))

# 120 kHz 1
# 2017 (only 1 cal) 24.56, -0.39
print('2017, 120 @ 1: ',24.56, -0.39)

# 2019
# Cal1: 25.36, -0.35
# Cal2: 24.46, -0.36
print('2019, 120 @ 1: ',round(10*np.log10(((10**(25.36/10))+(10**(24.46/10)))/2),3),round(10*np.log10(((10**(-.35/10))+(10**(-.36/10)))/2),3))

print(1-(10**((24.56 -0.39)/10))/(10**((24.933 -0.355)/10)))

2017, 120 @ .512:  24.79 -0.46
2019, 120 @ .512:  24.959 -0.45
0.04037843288965892
2017, 120 @ 1:  24.56 -0.39
2019, 120 @ 1:  24.933 -0.355
0.08966760032009014


In [190]:
# Net Openings
b = pd.read_csv('../data/catchData/2017_2019/AIERP_EventData.csv')
print('2017 vert opening +STD: ',b[b.SURVEY==201701].AVG_NET_VERT_OPENING.mean(),b[b.SURVEY==201701].AVG_NET_VERT_OPENING.std())
print('2017 hori opening +STD: ',b[b.SURVEY==201701].AVG_NET_HORI_OPENING.mean(),b[b.SURVEY==201701].AVG_NET_HORI_OPENING.std())
print('2019 vert opening +STD: ',b[b.SURVEY==201901].AVG_NET_VERT_OPENING.mean(),b[b.SURVEY==201901].AVG_NET_VERT_OPENING.std())
print('2019 hori opening +STD: ',b[b.SURVEY==201901].AVG_NET_HORI_OPENING.mean(),b[b.SURVEY==201901].AVG_NET_HORI_OPENING.std())
print('2017 HR mean, min, max :',b[b.SURVEY==201701].AVG_HEAD_ROPE_DEPTH.mean(),b[b.SURVEY==201701].AVG_HEAD_ROPE_DEPTH.min(),b[b.SURVEY==201701].AVG_HEAD_ROPE_DEPTH.max())
print('2019 HR mean, min, max :',b[b.SURVEY==201901].AVG_HEAD_ROPE_DEPTH.mean(),b[b.SURVEY==201901].AVG_HEAD_ROPE_DEPTH.min(),b[b.SURVEY==201901].AVG_HEAD_ROPE_DEPTH.max())

2017 vert opening +STD:  8.280166666666666 1.12605543084403
2017 hori opening +STD:  7.559833333333334 0.9346034410133652
2019 vert opening +STD:  7.533414634146343 0.5061650410493478
2019 hori opening +STD:  7.449062499999999 0.3429413509060847
2017 HR mean, min, max : 27.12212121212121 11.46 46.78
2019 HR mean, min, max : 34.98837209302326 13.27 227.9


In [222]:
# Number of CTD Stations
c = pd.read_csv('catchAnalysis/analysisFiles/dfStat.csv')
c = c[(~c.meanSa.isnull())&(~c.meanAcod.isnull())&(~c.meanPol.isnull())&(~c.meanCap.isnull())]
print('# CTD per year: ',len(c[c.Year == 2012]),len(c[c.Year == 2013]),len(c[c.Year == 2017]),len(c[c.Year == 2019]))

# CTD per year:  68 55 39 46


### Results

In [3]:
# CPUE
dfEvents = pd.read_csv('../data/catchData/2017_2019/AIERP_EventData.csv')
catch = pd.read_csv('../data/catchData/2017_2019/catchExport.csv')
events = pd.read_csv('../data/catchData/2017_2019/eventExport.csv')

dfCatch = catch[(catch.GEAR == 'Marinovich') & (catch.SPECIES_CODE <40000) & (catch.SURVEY==201701)].merge(dfEvents, how='left',  left_on=['SURVEY','CLAMS_EVENT_NUMBER'],   right_on=['SURVEY','EVENT_ID'])
dfCatch['CPUE'] = dfCatch.TOTAL_NUMBER_IN_HAUL/dfCatch.VOL_FILTERED
dfCatch = dfCatch.groupby(by=['CLAMS_EVENT_NUMBER']).sum()
print('2017 All fishes ',dfCatch.CPUE.mean())


dfCatch = catch[(catch.GEAR == 'Marinovich') & (catch.SPECIES_CODE ==21725) & (catch.SURVEY==201701)].merge(dfEvents, how='left',  left_on=['SURVEY','CLAMS_EVENT_NUMBER'],   right_on=['SURVEY','EVENT_ID'])
dfCatch['CPUE'] = dfCatch.TOTAL_NUMBER_IN_HAUL/dfCatch.VOL_FILTERED
dfCatch = dfCatch.groupby(by=['CLAMS_EVENT_NUMBER']).sum()
print('2017 All Arctic Cod ',dfCatch.CPUE.mean())

dfCatch = catch[(catch.GEAR == 'Marinovich') & (catch.SPECIES_CODE <40000) & (catch.SURVEY==201901)].merge(dfEvents, how='left',  left_on=['SURVEY','CLAMS_EVENT_NUMBER'],   right_on=['SURVEY','EVENT_ID'])
dfCatch['CPUE'] = dfCatch.TOTAL_NUMBER_IN_HAUL/dfCatch.VOL_FILTERED
dfCatch = dfCatch.groupby(by=['CLAMS_EVENT_NUMBER']).sum()
print('2019 All fishes ',dfCatch.CPUE.mean())


dfCatch = catch[(catch.GEAR == 'Marinovich') & (catch.SPECIES_CODE ==21725) & (catch.SURVEY==201901)].merge(dfEvents, how='left',  left_on=['SURVEY','CLAMS_EVENT_NUMBER'],   right_on=['SURVEY','EVENT_ID'])
dfCatch['CPUE'] = dfCatch.TOTAL_NUMBER_IN_HAUL/dfCatch.VOL_FILTERED
dfCatch = dfCatch.groupby(by=['CLAMS_EVENT_NUMBER']).sum()
print('2019 All Arctic Cod ',dfCatch.CPUE.mean())

2017 All fishes  0.24789253188085802
2017 All Arctic Cod  0.20975026751475034
2019 All fishes  0.03478787966128983
2019 All Arctic Cod  0.01788178450288881


In [257]:
# Lengths from catch
a = pd.read_csv('../data/catchData/2017_2019/specimen_complete_selectivity.csv')
import matplotlib.pyplot as plt
b = a[(a.SURVEY==201701)&(~a.CONSISTENT_LENGTH.isnull())].CONSISTENT_LENGTH.values
print('2017 length of 80 percentile: ',np.percentile(b,80))
b = a[(a.SURVEY==201901)&(~a.CONSISTENT_LENGTH.isnull())].CONSISTENT_LENGTH.values
print('2019 length of 80 percentile: ',np.percentile(b,80))

2017 length of 80 percentile:  7.3
2019 length of 80 percentile:  8.0


In [7]:
# Abundance and biomass from catch
a = pd.read_csv('../data/catchData/2017_2019/catchExport.csv')
a = a[(a.GEAR == 'Marinovich') & (a.NET_PARTITION == 'Codend')]
a = a[a.SURVEY == 201701]
a = a[(a.SPECIES_CODE >=1200)&(a.SPECIES_CODE < 60000)]
aJel = np.unique(a[(a.SPECIES_CODE > 40000) & (a.SPECIES_CODE  < 50000)].SPECIES_CODE.values)# jellyfish
aGad = np.unique( a[((a.SPECIES_CODE> 21700) & (a.SPECIES_CODE < 21750)) | (a.SPECIES_CODE == 1202)].SPECIES_CODE.values) # gadids    
aSan =  np.unique(a[(a.SPECIES_CODE >= 20202) & (a.SPECIES_CODE <= 20204)].SPECIES_CODE.values) # sand lance
aStic =  np.unique(a[(a.SPECIES_CODE >= 23800) & (a.SPECIES_CODE <= 23810)].SPECIES_CODE.values) # prickleback
aCap =  np.unique(a[(a.SPECIES_CODE== 23041)].SPECIES_CODE.values) # capelin
aHer =  np.unique(a[(a.SPECIES_CODE == 21110)].SPECIES_CODE.values) # herring
print('2017')
print('% fish abundance:', a[a.SPECIES_CODE.isin(np.concatenate([aGad,aSan,aStic,aCap,aHer]))].TOTAL_NUMBER_IN_HAUL.sum()/a.TOTAL_NUMBER_IN_HAUL.sum())
print('% Gad,cap,her abundance:', a[a.SPECIES_CODE.isin(np.concatenate([aGad,aCap,aHer]))].TOTAL_NUMBER_IN_HAUL.sum()/a.TOTAL_NUMBER_IN_HAUL.sum())
print('% fish weight:', a[a.SPECIES_CODE.isin(np.concatenate([aGad,aSan,aStic,aCap,aHer]))].TOTAL_WEIGHT_IN_HAUL.sum()/a.TOTAL_WEIGHT_IN_HAUL.sum())
print('% jelly abundance:', a[a.SPECIES_CODE.isin(aJel)].TOTAL_NUMBER_IN_HAUL.sum()/a.TOTAL_NUMBER_IN_HAUL.sum())
print('% jelly weight:', a[a.SPECIES_CODE.isin(aJel)].TOTAL_WEIGHT_IN_HAUL.sum()/a.TOTAL_WEIGHT_IN_HAUL.sum())
print('C. melanaster: ',a[a.SPECIES_CODE==40504].TOTAL_WEIGHT_IN_HAUL.sum()/a[a.SPECIES_CODE.isin(aJel)].TOTAL_WEIGHT_IN_HAUL.sum())

a = pd.read_csv('../data/catchData/2017_2019/catchExport.csv')
a = a[(a.GEAR == 'Marinovich') & (a.NET_PARTITION == 'Codend')]
a = a[a.SURVEY == 201901]
a = a[(a.SPECIES_CODE >=1200)&(a.SPECIES_CODE < 60000)]
aJel = np.unique(a[(a.SPECIES_CODE > 40000) & (a.SPECIES_CODE  < 50000)].SPECIES_CODE.values)# jellyfish
aGad = np.unique( a[((a.SPECIES_CODE> 21700) & (a.SPECIES_CODE < 21750)) | (a.SPECIES_CODE == 1202)].SPECIES_CODE.values) # gadids    
aSan =  np.unique(a[(a.SPECIES_CODE >= 20202) & (a.SPECIES_CODE <= 20204)].SPECIES_CODE.values) # sand lance
aStic =  np.unique(a[(a.SPECIES_CODE >= 23800) & (a.SPECIES_CODE <= 23810)].SPECIES_CODE.values) # prickleback
aCap =  np.unique(a[(a.SPECIES_CODE== 23041)].SPECIES_CODE.values) # capelin
aHer =  np.unique(a[(a.SPECIES_CODE == 21110)].SPECIES_CODE.values) # herring
print('2019')
print('% fish abundance:', a[a.SPECIES_CODE.isin(np.concatenate([aGad,aSan,aStic,aCap,aHer]))].TOTAL_NUMBER_IN_HAUL.sum()/a.TOTAL_NUMBER_IN_HAUL.sum())
print('% Gad,cap,her abundance:', a[a.SPECIES_CODE.isin(np.concatenate([aGad,aCap,aHer]))].TOTAL_NUMBER_IN_HAUL.sum()/a.TOTAL_NUMBER_IN_HAUL.sum())
print('% fish weight:', a[a.SPECIES_CODE.isin(np.concatenate([aGad,aSan,aStic,aCap,aHer]))].TOTAL_WEIGHT_IN_HAUL.sum()/a.TOTAL_WEIGHT_IN_HAUL.sum())
print('% jelly abundance:', a[a.SPECIES_CODE.isin(aJel)].TOTAL_NUMBER_IN_HAUL.sum()/a.TOTAL_NUMBER_IN_HAUL.sum())
print('% jelly weight:', a[a.SPECIES_CODE.isin(aJel)].TOTAL_WEIGHT_IN_HAUL.sum()/a.TOTAL_WEIGHT_IN_HAUL.sum())
print('C. melanaster: ',a[a.SPECIES_CODE==40504].TOTAL_WEIGHT_IN_HAUL.sum()/a[a.SPECIES_CODE.isin(aJel)].TOTAL_WEIGHT_IN_HAUL.sum())

2017
% fish abundance: 0.9863232129731636
% Gad,cap,her abundance: 0.9627595542267834
% fish weight: 0.16028538733154946
% jelly abundance: 0.007698121572572092
% jelly weight: 0.8383359547173879
C. melanaster:  0.6088457142338402
2019
% fish abundance: 0.9342907275666883
% Gad,cap,her abundance: 0.6369994922285663
% fish weight: 0.06770644265539377
% jelly abundance: 0.05539703561310407
% jelly weight: 0.93123636563934
C. melanaster:  0.5839014769756139


In [336]:
# Backscatter
df2012Summary = pd.read_csv('../data/acousticData/2012_2013/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2012_v3.csv')
df2013Summary = pd.read_csv('../data/acousticData/2012_2013/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2013_v3.csv')
df2017Summary = pd.read_csv('../data/acousticData/2017_2019/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2017.csv')
df2018Summary = pd.read_csv('../data/acousticData/2018/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2018.csv')
df2019Summary = pd.read_csv('../data/acousticData/2017_2019/Arctic_EIS_Acoustic_trawl_survey_alongtrack_summary_2019.csv')
print(np.mean(df2012Summary['Fish 38 kHz sA (m^2 nmi^-2)'][(df2012Summary.Latitude > 66)]))# & (df2012Summary.Latitude  < 71.85)]))
print(np.mean(df2013Summary['Fish 38 kHz sA (m^2 nmi^-2)'][(df2013Summary.Latitude > 66)]))# & (df2013Summary.Latitude  < 71.85)]))
print(np.mean(df2017Summary['Fish 38 kHz sA (m^2 nmi^-2)'][(df2017Summary.Latitude > 66)]))# & (df2017Summary.Latitude  < 71.85)]))
print(np.mean(df2018Summary['Fish 38 kHz sA (m^2 nmi^-2)'][(df2018Summary.Latitude > 66)]))# & (df2018Summary.Latitude  < 71.85)]))
print(np.mean(df2019Summary['Fish 38 kHz sA (m^2 nmi^-2)'][(df2019Summary.Latitude > 66)]))# & (df2019Summary.Latitude  < 71.85)]))
print('2017 is X times higher than the other years: ',1118.5/71.9,1118.5/257.5,1118.5/178.5,1118.5/354.8)

71.90508847228669
257.59256736688155
1118.5119704976917
178.53462277255952
354.8068172398032
2017 is X times higher than the other years:  15.556328233657856 4.343689320388349 6.266106442577031 3.152480270574972


In [5]:
a = pd.read_csv('../data/catchData/2017_2019/mbaCatchResults.csv')
a.SPECIES_CODE.unique()
#pollock
print('Pollock 2017:',a[(a.SURVEY==201701)&(a.SPECIES_CODE==21744)].NUMBERS.sum())
print('Pollock 2019:',a[(a.SURVEY==201901)&(a.SPECIES_CODE==21744)].NUMBERS.sum())
#arctic cod
print('Acod 2017:',a[(a.SURVEY==201701)&(a.SPECIES_CODE==21725)].NUMBERS.sum())
print('Acod 2019:',a[(a.SURVEY==201901)&(a.SPECIES_CODE==21725)].NUMBERS.sum())
# capelin
print('Capelin 2017:',a[(a.SURVEY==201701)&(a.SPECIES_CODE==23041)].NUMBERS.sum())
print('Capelin 2019:',a[(a.SURVEY==201901)&(a.SPECIES_CODE==23041)].NUMBERS.sum())

Pollock 2017: 74757212076.9931
Pollock 2019: 57404415222.94378
Acod 2017: 898100743353.7224
Acod 2019: 125555744574.5253
Capelin 2017: 16160487002.227179
Capelin 2019: 3739842338.6177306


In [16]:
specs = [21725, 21744, 21740, 21735, 21720]
print('2017 Gadids:',a[(a.SURVEY==201701)&(a.SPECIES_CODE.isin(specs))].NUMBERS.sum()/a[(a.SURVEY==201701)].NUMBERS.sum())
print('2019 Gadids:',a[(a.SURVEY==201901)&(a.SPECIES_CODE.isin(specs))].NUMBERS.sum()/a[(a.SURVEY==201901)].NUMBERS.sum())

print('2017 Arctic cod:',a[(a.SURVEY==201701)&(a.SPECIES_CODE==21725)].NUMBERS.sum()/a[(a.SURVEY==201701)].NUMBERS.sum())
print('2019 Arctic cod:',a[(a.SURVEY==201901)&(a.SPECIES_CODE==21725)].NUMBERS.sum()/a[(a.SURVEY==201901)].NUMBERS.sum())

print('2017 Pollock:',a[(a.SURVEY==201701)&(a.SPECIES_CODE.isin([21740,21744]))].NUMBERS.sum()/a[(a.SURVEY==201701)].NUMBERS.sum())
print('2019 Pollock:',a[(a.SURVEY==201901)&(a.SPECIES_CODE.isin([21740,21744]))].NUMBERS.sum()/a[(a.SURVEY==201901)].NUMBERS.sum())

2017 Gadids: 0.9838883646443122
2019 Gadids: 0.9641131437639759
2017 Arctic cod: 0.8953858684823652
2019 Arctic cod: 0.6516882495320803
2017 Pollock: 0.0745312279899934
2019 Pollock: 0.29801677432207296


In [155]:
#Abundances by survey by species
for surv in a.SURVEY.unique():
    for spec in a.SPECIES_CODE.unique():
        print(surv,spec,a[(a.SURVEY==surv)&(a.SPECIES_CODE==spec)].NUMBERS.sum()/10**10)

201701 23041 1.616048700222718
201701 21744 7.47572120769931
201701 21725 89.81007433537224
201701 21720 0.9413142382774418
201701 21735 0.46004876872346534
201701 21110 0.0
201701 21740 0.0
201901 23041 0.37398423386177304
201901 21744 5.740441522294378
201901 21725 12.55557445745253
201901 21720 0.08921857967994705
201901 21735 0.18837154481527138
201901 21110 0.31742012372610956
201901 21740 0.0012176257066017209


In [158]:
# lengths of arctic cod and pollock
df2012 = pd.read_csv('../data/catchData/2012_2013/Arctic_EIS_AT_survey_Arctic_cod_by_length_alongtrack_fish_per_m_squared_2012_v2.csv')
df2013 = pd.read_csv('../data/catchData/2012_2013/Arctic_EIS_AT_survey_Arctic_cod_by_length_alongtrack_fish_per_m_squared_2013_v2.csv')
df2017 = pd.read_csv('../data/catchData/2017_2019/Arctic_EIS_201701_21725.csv')
df2019 = pd.read_csv('../data/catchData/2017_2019/Arctic_EIS_201901_21725.csv')
print('2012 Acod:',sum(df2012[df2012.Latitude >=65.9].sum()[3:]/sum(df2012[df2012.Latitude >=65.9].sum()[3:])*np.arange(2,31,1)))
print('2013 Acod:',sum(df2013[df2013.Latitude >=65.9].sum()[3:]/sum(df2013[df2013.Latitude >=65.9].sum()[3:])*np.arange(2,31,1)))
print('2017 Acod:',sum(df2017.sum()[3:23]/sum(df2017.sum()[3:23])*np.arange(1,19,1)))
print('2019 Acod:',sum(df2019.sum()[3:23]/sum(df2019.sum()[3:23])*np.arange(1,21,1)))

#pollock
df2012 = pd.read_csv('../data/catchData/2012_2013/Arctic_EIS_AT_survey_pollock_by_length_alongtrack_fish_per_m_squared_2012_for_IERP.csv')
df2013 = pd.read_csv('../data/catchData/2012_2013/Arctic_EIS_AT_survey_pollock_by_length_alongtrack_fish_per_m_squared_2013_for_IERP.csv')
df2017 = pd.read_csv('../data/catchData/2017_2019/Arctic_EIS_201701_21744.csv')
df2019 = pd.read_csv('../data/catchData/2017_2019/Arctic_EIS_201901_21744.csv')
print('2012 pollock:',sum(df2012.sum()[3:23]/sum(df2012.sum()[3:23])*np.arange(2,22,1)))
print('2013 pollock:',sum(df2013.sum()[3:23]/sum(df2013.sum()[3:23])*np.arange(2,22,1)))
print('2017 pollock:',sum(df2017.sum()[3:23]/sum(df2017.sum()[3:23])*np.arange(1,19,1)))
print('2019 pollock:',sum(df2019.sum()[3:23]/sum(df2019.sum()[3:23])*np.arange(1,21,1)))

2012 Acod: 3.535163790827692
2013 Acod: 3.5679015478588365
2017 Acod: 4.408042066718216
2019 Acod: 4.808930212883881
2012 pollock: 4.978815338917016
2013 pollock: 6.872612340325574
2017 pollock: 5.107030595963149
2019 pollock: 5.068405358275401


### Proportion of acoustic backscatter attributed to the key species
```sql
select a.survey, b.event_id,a.interval,  a.layer,a.prc_nasc, c.species_code, c.sa_proportion
from 
(select * from integration_results) a
join
(select * from v_mean_sigma_by_interval_class) b
on a.interval = b.interval
and a.ship = b.ship
and a.survey = b.survey
join
(select * from scaling_key_sa_by_species) c
on b.event_id = c.event_id
and b.ship = c.ship
and b.survey = c.survey
where a.ship = 175 and (a.survey=201701 or a.survey = 201901)
order by a.interval, c.species_code
```

In [10]:
a = pd.read_csv('../data/acousticData/2017_2019/mbaSpeciesProp.csv')
a['NASC_SPECIES'] = a.PRC_NASC*a.SA_PROPORTION

c = a[a.SURVEY==201701]
b = c.groupby(by=['SPECIES_CODE']).sum().reset_index()
print('2017')
display(b.sort_values(by='NASC_SPECIES',ascending=False)[['SPECIES_CODE','NASC_SPECIES']].head(10))
specs = [21110, 23041, 21725, 21744, 21740, 21735, 21720]
b[b.SPECIES_CODE.isin(specs)].NASC_SPECIES.sum()/b.NASC_SPECIES.sum()
print('Prop key spec: ',b[b.SPECIES_CODE.isin(specs)].NASC_SPECIES.sum()/b.NASC_SPECIES.sum())
specs = [21725, 21744, 21740, 21735, 21720]
print('Prop gadid: ',b[b.SPECIES_CODE.isin(specs)].NASC_SPECIES.sum()/b.NASC_SPECIES.sum())

c = a[a.SURVEY==201901]
print('2019')
b = c.groupby(by=['SPECIES_CODE']).sum().reset_index()
display(b.sort_values(by='NASC_SPECIES',ascending=False)[['SPECIES_CODE','NASC_SPECIES']].head(10))
specs = [21110, 23041, 21725, 21744, 21740, 21735, 21720]
print('Prop key spec: ',b[b.SPECIES_CODE.isin(specs)].NASC_SPECIES.sum()/b.NASC_SPECIES.sum())
specs = [21725, 21744, 21740, 21735, 21720]
print('Prop gadid: ',b[b.SPECIES_CODE.isin(specs)].NASC_SPECIES.sum()/b.NASC_SPECIES.sum())

2017


,SPECIES_CODE,NASC_SPECIES
14,21725,2.121468e+06
16,21744,9.728182e+05
15,21735,2.225326e+05
13,21720,1.515828e+05
19,23041,8.660144e+04
25,23808,8.464975e+04
21,23801,2.520251e+04
8,21334,9.884667e+03
9,21368,5.670676e+03
20,23800,5.363116e+03


Prop key spec:  0.959912798031704
Prop gadid:  0.9365288980256332
2019


,SPECIES_CODE,NASC_SPECIES
21,21744,1.860306e+06
18,21725,8.055948e+05
19,21735,2.968549e+05
9,21110,1.710237e+05
31,23807,1.026548e+05
27,23041,8.184877e+04
17,21720,5.376915e+04
29,23800,3.123052e+04
20,21740,3.100961e+04
33,40504,2.907665e+04


Prop key spec:  0.927883722923627
Prop gadid:  0.8567905930082398


#### Total number of pollock found in catch north of the strait in 2012 and 2013 in the catch
```sql 
select b.eq_latitude, a.*
from
(select * from v_aeis_catch_summary_v2 where survey=2012001 and (species_code = 21740 or species_code = 21744)) a
join
(select * from v_event_data) b
on a.ship = b.ship and a.survey = b.survey and a.clams_event_number = b.event_id
order by eq_latitude
```

28 in 2012, 2 in 2017

### Temperature

In [6]:
dfStat = pd.read_csv('catchAnalysis/analysisFiles/dfStat.csv')

In [13]:
#dfStat[dfStat.Year == 2017].T_surf.mean()
print(dfStat[dfStat.Year == 2017].T_bot.max(),dfStat[dfStat.Year == 2017].T_bot.min())
print(dfStat[dfStat.Year == 2017].S_bot.max(),dfStat[dfStat.Year == 2017].S_bot.min())

6.577283333333334 -0.7174166666666667
34.73803333333333 31.094583333333336


#### Genetics

In [49]:
# This is not up to date...
a = pd.read_csv('C:/Users/robert.levine/Work/repositories/ArcticEISII/catchProcessing/code/specimenCorrections/geneticID/data/gadidsUpdatedLen.csv')
a = a[(a.GEAR=='Marinovich')&(a.PARTITION=='Codend')]
print('2017 genetics done: ',len(a[(a.SURVEY==201701) & ((a.SPECIES_ID_METHOD=='genetics_changed')|(a.SPECIES_ID_METHOD=='genetics_confirmed'))]))
print('2017 all specimen: ',len(a[(a.SURVEY==201701)]))

print('2019 genetics done: ',len(a[(a.SURVEY==201901) & ((a.SPECIES_ID_METHOD=='genetics_changed')|(a.SPECIES_ID_METHOD=='genetics_confirmed'))]))
print('2019 all specimen: ',len(a[(a.SURVEY==201901)]))

2017 genetics done:  755
2017 all specimen:  2244
2019 genetics done:  2983
2019 all specimen:  5654


```sql select count(*) from v_aeis_specimen
where survey = 201701 and
gear = 'Marinovich'
and net_partition = 'Codend'
and species_code < 40000

3392

and species_code > 40000

1211

select count(*) from v_aeis_specimen
where survey = 201701 and
gear = 'Marinovich'
and net_partition = 'Codend'
and (species_code > 21700
and species_code < 21800
or species_code = 1202)
order by species_code

2244

select count(*) from v_aeis_specimen
where survey = 201701 and
gear = 'Marinovich'
and net_partition = 'Codend'
and (species_id_method = 'genetics_confirmed'
or species_id_method = 'genetics_changed')

894

select count(*) from v_aeis_specimen
where survey = 201701 and
gear = 'Marinovich'
and net_partition = 'Codend'
and species_id_method = 'genetics_model_assigned'

1350

select count(*) from v_aeis_specimen
where survey = 201701 and
gear = 'Marinovich'
and net_partition = 'Codend'
and species_id_method = 'field_id'

0


select count(*) from v_aeis_specimen
where survey = 201901 and
gear = 'Marinovich'
and net_partition = 'Codend'
and species_code < 40000

9124

and species_code > 40000

751

select count(*) from v_aeis_specimen
where survey = 201901 and
gear = 'Marinovich'
and net_partition = 'Codend'
and (species_code > 21700
and species_code < 21800
or species_code = 1202)
order by species_code

5676


select count(*) from v_aeis_specimen
where survey = 201901 and
gear = 'Marinovich'
and net_partition = 'Codend'
and (species_id_method = 'genetics_confirmed'
or species_id_method = 'genetics_changed')

3155

select count(*) from v_aeis_specimen
where survey = 201901 and
gear = 'Marinovich'
and species_id_method = 'genetics_model_assigned'

2488

select count(*) from v_aeis_specimen
where survey = 201901 and
gear = 'Marinovich'
and net_partition = 'Codend'
and species_id_method = 'field_id'

11
```